In [21]:
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy.optimize import minimize

In [52]:
### set up
lai_emulator = tf.saved_model.load('./emulators/laimax_gmean/')
gpp_emulator = tf.saved_model.load('./emulators/gpp_gmean/')
biomass_emulator = tf.saved_model.load('./emulators/biomass_gmean/')

# input params
lhckey = './utils/lhc220926.txt'
ppe_params = pd.read_csv(lhckey).drop(columns='member')

tuning_params = ['FUN_fracfixers','froot_leaf','jmaxb0','jmaxb1','kmax','leafcn','medlynintercept','medlynslope','slatop','wc2wjb0']
num_params = len(tuning_params)


In [53]:
# set up perfect model experiment; try to recover a known parameter set
target_paramset = ppe_params.iloc[13]

target_lai = lai_emulator.compiled_predict_f(target_paramset.values.reshape(1, -1))[0].numpy().flatten()[0]
target_gpp = gpp_emulator.compiled_predict_f(test_paramset.values.reshape(1, -1))[0].numpy().flatten()[0]
target_biomass = biomass_emulator.compiled_predict_f(test_paramset.values.reshape(1, -1))[0].numpy().flatten()[0]

### Scipy optimize

In [56]:
def objective(x):
    sample = target_paramset.copy()
    sample[tuning_params] = x

    lai = lai_emulator.compiled_predict_f(sample.values.reshape(1, -1))[0].numpy().flatten()[0]
    gpp = gpp_emulator.compiled_predict_f(sample.values.reshape(1, -1))[0].numpy().flatten()[0]
    biomass = biomass_emulator.compiled_predict_f(sample.values.reshape(1, -1))[0].numpy().flatten()[0]

    # Scaled squared errors
    loss_lai = ((lai - target_lai) / target_lai)**2
    loss_gpp = ((gpp - target_gpp) / target_gpp)**2
    loss_biomass = ((biomass - target_biomass) / target_biomass)**2

    return loss_lai + loss_gpp + loss_biomass

In [61]:
# Initial guess
x0 = np.random.rand(num_params)

# Bounds for each parameter in [0, 1]
bounds = [(0, 1)] * num_params

# Run optimization with L-BFGS-B
result = minimize(objective, x0, method='L-BFGS-B', bounds=bounds)

# Run optimization with Nelder-Mead methodology
#result = minimize(objective, x0, method='Nelder-Mead')

In [62]:
# check
recovered_sample = target_paramset.copy()
recovered_sample[tuning_params] = result.x

recovered_lai = lai_emulator.compiled_predict_f(recovered_sample.values.reshape(1, -1))[0].numpy().flatten()[0]
recovered_gpp = gpp_emulator.compiled_predict_f(recovered_sample.values.reshape(1, -1))[0].numpy().flatten()[0]
recovered_biomass = biomass_emulator.compiled_predict_f(recovered_sample.values.reshape(1, -1))[0].numpy().flatten()[0]

print("True LAI:", target_lai, "Recovered LAI:", recovered_lai)
print("True GPP:", target_gpp, "Recovered GPP:", recovered_gpp)
print("True Biomass:", target_biomass, "Recovered Biomass:", recovered_biomass)
print("Recovered - true parameters:", result.x - target_paramset[tuning_params])


True LAI: 1.6622059365545285 Recovered LAI: 1.7416256751908736
True GPP: 152.19816441369932 Recovered GPP: 140.11932314245342
True Biomass: 806.8224757298698 Recovered Biomass: 786.5253101254951
Recovered - true parameters: FUN_fracfixers     0.934146
froot_leaf         0.216624
jmaxb0             0.134356
jmaxb1            -0.317099
kmax              -0.076611
leafcn             0.071293
medlynintercept    0.387956
medlynslope       -0.838060
slatop            -0.341671
wc2wjb0            0.084596
Name: 13, dtype: float64


### Tensorflow ADAM optimization

In [64]:
# Convert target values to tensors for use in loss
target_tensor = tf.constant([target_lai, target_gpp, target_biomass], dtype=tf.float32)

In [63]:
# Initialize optimization
x = tf.Variable(np.random.rand(num_params), dtype=tf.float32)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.05)

In [78]:
# Optimization loop
def get_loss_and_grads():
    with tf.GradientTape() as tape:
        sample_array = target_paramset.values.copy()
        for i, param in enumerate(tuning_params):
            idx = ppe_params.columns.get_loc(param)
            sample_array[idx] = x[i]

        full_input = tf.convert_to_tensor(sample_array.reshape(1, -1), dtype=tf.float64)

        lai = lai_emulator.compiled_predict_f(full_input)[0][0, 0]
        gpp = gpp_emulator.compiled_predict_f(full_input)[0][0, 0]
        biomass = biomass_emulator.compiled_predict_f(full_input)[0][0, 0]

        preds = tf.stack([lai, gpp, biomass])
        loss = tf.reduce_sum(tf.square((preds - target_tensor) / target_tensor))
    grads = tape.gradient(loss, [x])
    return loss, grads

In [79]:
for step in range(200):
    loss, grads = get_loss_and_grads()
    optimizer.apply_gradients(zip(grads, [x]))
    x.assign(tf.clip_by_value(x, 0.0, 1.0))  # enforce bounds manually
    if step % 20 == 0:
        print(f"Step {step}: Loss = {loss.numpy():.6f}")

InvalidArgumentError: cannot compute Sub as input #1(zero-based) was expected to be a double tensor but is a float tensor [Op:Sub] name: 